In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from time import sleep
from selenium.webdriver.common.by import By
from fuzzywuzzy import fuzz
import re
import pickle
import logging
import json
from time import sleep
from pymongo import MongoClient
from lib.netflix_scraper import NetflixSession

def get_movies_collection():
    client = MongoClient('localhost', 27017)
    db = client.moodvies
    return db['movies']

/Users/achadfried/dev/Envs/jupyter3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
session = NetflixSession()

In [7]:
session.load_cookies_into_browser()
session.load_streaming_movies_from_page(search_url='https://www.netflix.com/browse/genre/34399')

In [14]:
movie_collection = get_movies_collection()
movie_collection.insert_many(session.movies)

BulkWriteError: batch op errors occurred

In [13]:
# session.streaming_movies[0]    
options = ChromeOptions()
options.add_argument('--headless')

In [14]:
driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=options)

In [15]:
driver.get('https://www.netflix.com/NotFound?prev=http%3A%2F%2Fwww.netflix.com%2F404')
for i in range(len(session.cookies)):
    driver.add_cookie(session.cookies[i])

In [11]:
search_url = 'https://www.netflix.com/browse/genre/34399'
delay = 3
movies = {}
delay = 20
movies = {}
moods = []
driver.get(search_url)

In [7]:
html = driver.page_source

In [11]:
results_page = BeautifulSoup(html, "html.parser")

In [12]:
result_list = results_page.find_all(attrs={"class": "slider-refocus"})

In [310]:
html = driver.page_source
results_page = BeautifulSoup(html, "html.parser")
links = results_page.find_all('a', href=True)
moods = [link.text for link in links if 'genre' in link.attrs['href']]
moods


['TV Shows',
 'Movies',
 'Originals',
 'Recently Added',
 'Docs & Reality TVExplore All',
 'International DramasExplore All',
 'Critically-acclaimed MoviesExplore All',
 'NETFLIX ORIGINALSExplore All',
 'Critically-acclaimed Auteur CinemaExplore All',
 'Sports DocumentariesExplore All']

In [16]:
from selenium.webdriver import ActionChains
from time import sleep
search_url = 'https://www.netflix.com/browse/genre/34399'
driver.get(search_url)
html = driver.page_source
results_page = BeautifulSoup(html, "html.parser")
available_movies = results_page.find_all(attrs={"class": "slider-refocus"})
movies = []
for result in set(available_movies):
    try:
        title = result.find(attrs={"class": "fallback-text"}).string
        element = driver.find_element_by_partial_link_text(title)
        ActionChains(driver).move_to_element(element).click().perform();
        # sleep(2)
        WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CLASS_NAME, 'meta-lists')))
        # detail_element = driver.find_element_by_id("tab-ShowDetails")
        # ActionChains(driver).move_to_element(detail_element).click().perform();
        # WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'detailsTags')))
        # mood_element = driver.find_element_by_class_name("detailsTags")
        # ActionChains(driver).move_to_element(mood_element).perform()
        html = driver.page_source
        results_page = BeautifulSoup(html, "html.parser")
        links = results_page.find_all('a', href=True)

        watch_url = [link.attrs['href'] for link in links if 'aria-label' in link.attrs and link.attrs['aria-label'] == 'Play'][0]
        genres = [link.text for link in links if 'genre' in link.attrs['href']]
        print(results_page.find_all(attrs={"class": "synopsis"}))
        synopsis = results_page.find_all(attrs={"class": "synopsis"})[0]
        
        movie = {}
        movie['title'] = title
        movie['genres'] = genres
        movie['watch_url'] = watch_url
        movie['synopsis'] = synopsis.text
        movies.append(movie)
        print(title)
        print(synopsis.text)
        print(watch_url)
    except Exception as e:
        print(e)
# # genre_elements = driver.find_elements_by_xpath('//a[contains(@href, "/browse/genre")]')
# # [mood.text for mood in genre_elements]

[<div class="synopsis">During a tense period in history, King George VI struggles to communicate to the public and seeks help from speech therapist Lionel Logue.</div>]
The King's Speech
During a tense period in history, King George VI struggles to communicate to the public and seeks help from speech therapist Lionel Logue.
/watch/70135893?trackId=253840196&tctx=0%2C4%2C9df3e617-b7f2-4781-92e1-b2b3662d208c-26741661%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_125145806X28X3979X1528906838967%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_ROOT
[<div class="synopsis">During a tense period in history, King George VI struggles to communicate to the public and seeks help from speech therapist Lionel Logue.</div>]
Amelie
During a tense period in history, King George VI struggles to communicate to the public and seeks help from speech therapist Lionel Logue.
/watch/70135893?trackId=251477752&tctx=0%2C0%2C69d2ba7c-cec0-47b1-9b54-e992a55d85e7-27041712%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_125145806X28X3979X1

[<div class="synopsis">Amid the Irish and Italian rivalry that dominated New York City in the mid-1800s, young Amsterdam Vallon hunts down his father's hateful murderer.</div>]
Gangs of New York
Amid the Irish and Italian rivalry that dominated New York City in the mid-1800s, young Amsterdam Vallon hunts down his father's hateful murderer.
/watch/60021787?trackId=250311567&tctx=2%2C3%2C9df3e617-b7f2-4781-92e1-b2b3662d208c-26741661%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_125145808X10XX1528906838967%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_ROOT
[<div class="synopsis">Amid the Irish and Italian rivalry that dominated New York City in the mid-1800s, young Amsterdam Vallon hunts down his father's hateful murderer.</div>]
The Magic Pill
Amid the Irish and Italian rivalry that dominated New York City in the mid-1800s, young Amsterdam Vallon hunts down his father's hateful murderer.
/watch/60021787?trackId=253035256&tctx=0%2C0%2C060e095e-18d4-46fe-bd9a-cb17bf750be6-26829048%2C6aabcee8-b990-4a0a-a

Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Alex Strangelove"}
  (Session info: headless chrome=66.0.3359.181)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.12.6 x86_64)

Message: no such element: Unable to locate element: {"method":"partial link text","selector":"180° South"}
  (Session info: headless chrome=66.0.3359.181)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.12.6 x86_64)

[<div class="synopsis">This gripping thriller about Cold War espionage follows an English spy as he returns to MI-6 under suspicion that he's become a Soviet operative.</div>]
Icarus
This gripping thriller about Cold War espionage follows an English spy as he returns to MI-6 under suspicion that he's become a Soviet operative.
/watch/70202128?trackId=253035256&tctx=0%2C0%2Cfa0c7308-09cc-4257-8ffd-92ed9f05ce75-27033268%2C6aabcee8-b990-4a0a-aaf

Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Anon"}
  (Session info: headless chrome=66.0.3359.181)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.12.6 x86_64)

[<div class="synopsis">An argument about fidelity sends a married physician reeling into the night, where he finds himself in a surreal succession of sexual encounters.</div>]
Bombshell: The Hedy Lamarr Story
An argument about fidelity sends a married physician reeling into the night, where he finds himself in a surreal succession of sexual encounters.
/watch/5670434?trackId=251125819&tctx=8%2C1%2C9df3e617-b7f2-4781-92e1-b2b3662d208c-26741661%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_125145814X28X801360X1528906838967%2C6aabcee8-b990-4a0a-aafc-125f7baf2b45_ROOT
Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Ibiza"}
  (Session info: headless chrome=66.0.3359.181)
  (Driver info: ch

In [53]:
available_movies = results_page.find_all(attrs={"class": "slider-refocus"})
[ result.find(attrs={"class": "fallback-text"}).string for result in available_movies]

['Thor: Ragnarok',
 'Coco',
 'L.A. Confidential',
 "The King's Speech",
 'Heat',
 'Casino',
 'The Godfather',
 'Bombshell: The Hedy Lamarr Story',
 'The Departed',
 'Blue Jasmine',
 "Ali's Wedding",
 'White Right: Meeting the Enemy',
 'Ibiza',
 'Thor: Ragnarok',
 'American Experience: Tesla',
 'Oklahoma City',
 'American Experience: The Race Underground',
 'The Truth About Alcohol',
 'Prince Philip: The Plot to Make a King',
 "Secrets of Her Majesty's Secret Service",
 'American Experience: Rachel Carson',
 'The 40-Year-Old Virgin',
 'Kill Bill: Vol. 2',
 'The Godfather: Part II',
 'Michael Clayton',
 'Gangs of New York',
 'Tinker, Tailor, Soldier, Spy',
 'Burn After Reading',
 'The Godfather',
 'Jackie Brown',
 'Lincoln',
 'Dallas Buyers Club',
 'Spotlight',
 'Dead Poets Society',
 'The Departed',
 'Ibiza',
 'The Kissing Booth',
 'Icarus',
 'Anon',
 'The Week Of',
 'Alex Strangelove',
 'Mudbound',
 'Tropic Thunder',
 'Shrek',
 'Moonrise Kingdom',
 'Meet the Parents',
 'Midnight in Par

In [15]:
# watch_url = [link.text for link in links if link.attrs['aria-label'] == 'Play']
links[0].attrs

{'aria-label': 'Netflix',
 'class': ['logo', 'icon-logoUpdate'],
 'data-reactid': '7',
 'href': '/browse'}

In [20]:
html = driver.page_source
results_page = BeautifulSoup(html, "html.parser")
synopsis = results_page.find_all(attrs={"class": "synopsis"})[0]
synopsis.text

"To take down Boston's Irish Mafia, the police send in one of their own men to infiltrate it, not realizing the syndicate has done the same thing."

In [234]:
# html = driver.page_source
# results_page = BeautifulSoup(html, "html.parser")
# links = results_page.find_all('a', href=True)
# moods = []
# for link in links:
#     if 'genre' in link.attrs['href']:
#         moods.append(link.text)
# moods

In [235]:
# mood_element = driver.find_element_by_class_name("detailsTags")
# ActionChains(driver).move_to_element(mood_element).perform()
# genre_elements = driver.find_elements_by_xpath('//a[contains(@href, "/browse/genre/")]')
# [mood.text for mood in genre_elements]

In [236]:
# genre_elements = driver.find_elements_by_xpath('//a[contains(@href, "/browse/genre/")]')
#         detail_element = driver.find_element_by_id("tab-ShowDetails")
#         ActionChains(driver).move_to_element(detail_element).click().perform();
#         WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'detailsTags')))
#         mood_element = driver.find_element_by_class_name("detailsTags")
#         ActionChains(driver).move_to_element(mood_element).perform()
#     driver.get(search_url)
#     sleep(2)

In [13]:
movies = {}
driver.get(search_url)
for result in result_list[:10]:
    title = result.find(attrs={"class": "fallback-text"}).string
    try:
        title_element = driver.find_element_by_partial_link_text(title)
        ActionChains(driver).move_to_element(title_element).click().perform();
#         sleep(1)
        WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CLASS_NAME, 'meta-lists')))

        html = driver.page_source
        results_page = BeautifulSoup(html, "html.parser")
        links = results_page.find_all('a', href=True)
        moods = [link.text for link in links if 'genre' in link.attrs['href']]
        print(moods)
        movies[title] = {}
        movies[title]['moods'] = moods

    except Exception as e:
        print(title)
        print(e)
        continue

NameError: name 'result_list' is not defined

In [338]:
len(movies)
movies
# len(result_list)

{'Bombshell: The Hedy Lamarr Story': {'moods': ['TV Shows',
   'Movies',
   'Originals',
   'Recently Added',
   'Critically-acclaimed MoviesExplore All',
   'Documentaries',
   'Biographical Documentaries',
   'Historical Documentaries',
   'Inspiring',
   'Sports DocumentariesExplore All',
   'Understated MoviesExplore All',
   'NETFLIX ORIGINALSExplore All',
   'Award-winning MoviesExplore All',
   'ComediesExplore All']},
 'Casino': {'moods': ['TV Shows',
   'Movies',
   'Originals',
   'Recently Added',
   'Critically-acclaimed MoviesExplore All',
   'Sports DocumentariesExplore All',
   'Understated MoviesExplore All',
   'Action & Adventure',
   'Action Thrillers',
   'Crime Action & Adventure',
   'Violent',
   'NETFLIX ORIGINALSExplore All',
   'Award-winning MoviesExplore All',
   'ComediesExplore All']},
 'Coco': {'moods': ['TV Shows',
   'Movies',
   'Originals',
   'Recently Added',
   'Critically-acclaimed MoviesExplore All',
   'Children & Family Movies',
   'Movies for 

In [60]:
for result in result_list:
    driver.find_element_by_partial_link_text('')

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml"><head><title>Netflix</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="google" value="notranslate" /><link type="text/css" rel="stylesheet" href="https://codex.nflxext.com/%5E2.0.0/truthBundle/webui/0.0.1-shakti-css-0d8ef554/css/css/pages%7CakiraClient.less/1/0Q6KczJTXRLPWZ/none/true/none" /><link rel="shortcut icon" href="https://assets.nflxext.com/us/ffe/siteui/common/icons/nficon2016.ico" /><script src="https://www.gstatic.com/cv/js/sender/v1/cast_sender.js"></script><script async="" id="player-core-js" src="https://assets.nflxext.com/en_us/ffe/player/html/cadmium-playercore-6.0009.427.011.js"></script><script src="chrome-extension://pkedcjkdefgpdelpbcmbmeomcjbeemfm/cast_sender.js"></script><script src="chrome-extension://enhhojjnijigcajfphajepfemndkmdlo/cast_sender.js"></script></head><body sty

In [61]:
driver.find_element_by_id("tab-ShowDetails").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"id","selector":"tab-ShowDetails"}
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.12.6 x86_64)


In [62]:
html = driver.page_source

In [63]:
html

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml"><head><title>Netflix</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="google" value="notranslate" /><link type="text/css" rel="stylesheet" href="https://codex.nflxext.com/%5E2.0.0/truthBundle/webui/0.0.1-shakti-css-0d8ef554/css/css/pages%7CakiraClient.less/1/0Q6KczJTXRLPWZ/none/true/none" /><link rel="shortcut icon" href="https://assets.nflxext.com/us/ffe/siteui/common/icons/nficon2016.ico" /><script src="https://www.gstatic.com/cv/js/sender/v1/cast_sender.js"></script><script async="" id="player-core-js" src="https://assets.nflxext.com/en_us/ffe/player/html/cadmium-playercore-6.0009.427.011.js"></script><script src="chrome-extension://pkedcjkdefgpdelpbcmbmeomcjbeemfm/cast_sender.js"></script><script src="chrome-extension://enhhojjnijigcajfphajepfemndkmdlo/cast_sender.js"></script></head><body sty

In [64]:
moods = driver.find_elements_by_xpath('//a[contains(@href, "/browse/genre/")]');

In [65]:
tags = [mood.text for mood in moods]

In [66]:
tags

['TV Shows',
 'Movies',
 'Originals',
 'Recently Added',
 'NETFLIX ORIGINALS',
 'Critically-acclaimed Movies',
 'Recently Added',
 'Crime Movies',
 'Romantic Movies',
 'Dramas based on Real Life']